In [1]:
import json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

In [6]:
with open('/Users/tianyixia/dev/magisim/data/AllCards.json') as data_file: 
    data = json.load(data_file)

In [91]:
df = pd.DataFrame.from_dict(data, orient='index')

In [92]:
from gensim.models.doc2vec import *
import nltk
from nltk.corpus import stopwords
import re

In [93]:
def process_txt(text):
    return [re.sub(r'[^A-Za-z0-9{}+-?]+', '',x) for x in str(text).lower().split()
                 if x not in stopwords.words('english')]

In [94]:
def get_train_data_stream():
    for row_, card_ in df.iterrows():
        if (row_ % 1000 == 0):
            print("Now row", row_, "out of", df.shape[0])
        txt_ = process_txt(card_["text"])
        yield LabeledSentence(txt_, [card_["name"].lower()])

train_data = list(get_train_data())

TypeError: not all arguments converted during string formatting

In [102]:
model = Doc2Vec(alpha=0.025, min_alpha=0.025, workers=3)

In [103]:
model.build_vocab(train_data)

In [104]:
for epoch in range(10):
    model.train(train_data)
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay
    print("epoch {} done".format(epoch))

epoch 0 done
epoch 1 done
epoch 2 done
epoch 3 done
epoch 4 done
epoch 5 done
epoch 6 done
epoch 7 done
epoch 8 done
epoch 9 done


In [107]:
model.save("/Users/tianyixia/dev/mtg/saved_model")

In [68]:
"Lightning Bolt"

'Lightning Bolt'

In [109]:
def get_row(name_):
    index = -1
    for i_, card in df.iterrows():
        if name_ == card["name"]:
            index = i_
            break
    if index == -1:
        print("card not found")
        return
    return index
    
def get_similar_card(name_, topn=10):
    return [x[0] for x in model.docvecs.most_similar(name_.lower(), topn=topn)]

In [111]:
get_similar_card("lightning bolt", 5)

['lightning blast',
 'goblin mountaineer',
 'segovian leviathan',
 'yavimaya barbarian',
 'cliff threader']